In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
base_path = '../../'
sys.path.append(base_path)

In [3]:
from src.dgl_flow_field_dataset import DGLSurfaceFlowFieldDataset
from src.pyvista_flow_field_dataset import PyvistaFlowFieldDataset
ds_pv = PyvistaFlowFieldDataset.load_from_huggingface("datasets/pyvista",num_samples=3)
#ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'),ds_pv)
ds_dgl = DGLSurfaceFlowFieldDataset(os.path.join(base_path,'datasets/dgl_surface'))

Loaded 3 samples from 'datasets/pyvista'.


In [4]:
ds_pv[1].surface_data[0][1]

UnstructuredGrid (0x710ab8736680)
  N Cells:    1002
  N Points:   1042
  X Bounds:   8.316e-02, 1.167e-01
  Y Bounds:   9.825e-03, 4.827e-02
  Z Bounds:   0.000e+00, 1.500e-02
  N Arrays:   12

In [5]:
ds_pv[0].get_surface_points(block_index=0)

pyvista_ndarray([[0.498     , 0.        , 0.        ],
                 [0.5       , 0.002     , 0.        ],
                 [0.496     , 0.        , 0.        ],
                 ...,
                 [0.47394846, 0.08794846, 0.        ],
                 [0.474     , 0.058     , 0.02      ],
                 [0.47194449, 0.08594449, 0.        ]])

In [5]:
ds_pv[1].plot_volume("Pressure")

2025-05-26 07:03:39.888 (   8.621s) [    7627E4179740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x6249683bfe60): Skipping BC_t node: BC_t type 'BCInflow' not supported yet.
2025-05-26 07:03:39.888 (   8.621s) [    7627E4179740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x6249683bfe60): Skipping BC_t node: BC_t type 'BCSymmetryPlane' not supported yet.
2025-05-26 07:03:39.888 (   8.622s) [    7627E4179740]      vtkCGNSReader.cxx:4268  WARN| vtkCGNSReader (0x6249683bfe60): Skipping BC_t node: BC_t type 'BCTunnelOutflow' not supported yet.
/nfs/homedirs/peo/flow_field_dataset/.venv/lib/python3.11/site-packages/pyvista/plotting/plotter.py:159: UserWarning: 
This system does not appear to be running an xserver.
PyVista will likely segfault when rendering.

Try starting a virtual frame buffer with xvfb, or using
  ``pyvista.start_xvfb()``

  warnings.warn(
ERROR:root:bad X server connection. DISPLAY=


: 

In [6]:
ds_dgl[0]

Graph(num_nodes=34954, num_edges=136888,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})

In [7]:
from dgl.dataloading import GraphDataLoader
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataloader = GraphDataLoader(ds_dgl, batch_size=1, shuffle=True)
for batch in dataloader:
    print(batch)
    

Graph(num_nodes=34954, num_edges=136888,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})
Graph(num_nodes=35350, num_edges=138574,
      ndata_schemes={'BodyID': Scheme(shape=(), dtype=torch.int32), 'CellArea': Scheme(shape=(), dtype=torch.float32), 'Normal': Scheme(shape=(3,), dtype=torch.float32), 'ShearStress': Scheme(shape=(3,), dtype=torch.float32), 'Position': Scheme(shape=(3,), dtype=torch.float32), 'Temperature': Scheme(shape=(), dtype=torch.float32), 'Pressure': Scheme(shape=(), dtype=torch.float32)}
      edata_schemes={'dx': Scheme(shape=(3,), dtype=torch.float32)})
Graph(num_no

In [8]:
import dgl


def get_node_edge_X(graph: dgl.DGLGraph):
    node_X = torch.cat([graph.ndata["Position"], graph.ndata["Normal"]],dim=1)
    edge_X = torch.cat([graph.edata["dx"]],dim=1)
    return node_X, edge_X

def get_node_Y(graph: dgl.DGLGraph):
    return torch.cat([graph.ndata["Pressure"].unsqueeze(1),graph.ndata["Temperature"].unsqueeze(1),graph.ndata['ShearStress']],dim=1)

def set_graph_features(graph: dgl.DGLGraph, node_X, edge_X, node_Y):
    graph.ndata["Position"] = node_X[:,:3]
    graph.ndata["Normal"] = node_X[:,3:]
    graph.edata["dx"] = edge_X
    graph.ndata["Pressure"] = node_Y[:,0]
    graph.ndata["Temperature"] = node_Y[:,1]
    graph.ndata["ShearStress"] = node_Y[:,2:]
g=ds_dgl[0]
g_cp=g.clone()
ndx, edx = get_node_edge_X(g)
ndy = get_node_Y(g)
set_graph_features(g_cp, ndx, edx, ndy)
assert torch.allclose(g_cp.ndata["Position"], g.ndata["Position"])
assert torch.allclose(g_cp.ndata["Normal"], g.ndata["Normal"])
assert torch.allclose(g_cp.edata["dx"], g.edata["dx"])
assert torch.allclose(g_cp.ndata["Pressure"], g.ndata["Pressure"])
assert torch.allclose(g_cp.ndata["Temperature"], g.ndata["Temperature"])
assert torch.allclose(g_cp.ndata["ShearStress"], g.ndata["ShearStress"])
num_node_features = ndx.shape[1]
num_edge_features = edx.shape[1]
num_node_labels = ndy.shape[1]
print("Node X: ",ndx, ndx.shape)
print("Edge X: ",edx, edx.shape)
print("Node Y: ",ndy, ndy.shape)

Node X:  tensor([[-1.6428e+00,  1.4476e+00,  1.0175e+00,  3.2021e-03, -1.6390e-02,
          1.1908e+00],
        [-1.6295e+00,  1.4476e+00,  1.0175e+00,  3.2021e-03, -1.6390e-02,
          1.1908e+00],
        [-1.6428e+00,  1.3890e+00,  1.0175e+00,  3.2021e-03, -1.6390e-02,
          1.1908e+00],
        ...,
        [ 1.6866e+00, -1.3653e+00, -1.1289e+00,  4.2293e+00, -1.6390e-02,
          5.4273e-03],
        [ 1.6866e+00, -1.4239e+00, -9.3531e-01,  4.2293e+00, -1.6390e-02,
          5.4273e-03],
        [ 1.6866e+00, -1.4239e+00, -1.1300e+00,  4.2293e+00, -1.6390e-02,
          5.4273e-03]]) torch.Size([34954, 6])
Edge X:  tensor([[ 1.4469e+00,  6.1278e-10,  1.2620e-09],
        [-8.7035e-11, -1.6235e+00,  1.2620e-09],
        [-1.4469e+00,  6.1278e-10,  1.2620e-09],
        ...,
        [-8.7035e-11,  2.7059e-01, -1.2879e+00],
        [-8.7035e-11,  1.6235e+00,  1.7211e-02],
        [-8.7035e-11,  6.1278e-10,  3.0032e+00]]) torch.Size([136888, 3])
Node Y:  tensor([[ 3.6205e+00, 

In [9]:
from modulus.models.meshgraphnet import MeshGraphNet
model = MeshGraphNet(
    input_dim_nodes=num_node_features,
    input_dim_edges=num_edge_features,
    output_dim=num_node_labels,
    aggregation='sum',
    hidden_dim_edge_encoder=64,
    hidden_dim_node_encoder=64,
    hidden_dim_processor=64,
    hidden_dim_node_decoder=64
)
model=model.to(device)
from torch.optim import Adam
from torch.optim.lr_scheduler import LambdaLR
from torch.amp import GradScaler
optimizer = Adam(model.parameters(), lr=1e-4)
scheduler = LambdaLR(optimizer, lr_lambda=lambda epoch: 0.99985 ** epoch)
scaler = GradScaler(device=device)

In [10]:

from modulus.launch.utils import save_checkpoint, load_checkpoint
checkpoint_path = 'checkpoints'
os.makedirs(checkpoint_path,exist_ok=True)
epoch_init = load_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,device=device)
num_epochs = 400
model.to(device)
for epoch in range(epoch_init,num_epochs):
    total_loss = 0
    for batch in dataloader:
        batch = batch.to(device)
        node_X, edge_X = get_node_edge_X(batch)
        node_Y = get_node_Y(batch)
        node_Y_pred = model(node_X,edge_X,batch)
        batch_pred_graph = batch.clone()
        set_graph_features(batch_pred_graph, node_X, edge_X, node_Y_pred)
        agg_force_pred = ds_dgl.compute_aggregate_force(batch_pred_graph)
        agg_force = ds_dgl.compute_aggregate_force(batch)
        #print('Agg force pred: ',format_vector(agg_force_pred.tolist()),' Agg force: ',format_vector(agg_force.tolist()))
        loss = torch.nn.functional.mse_loss(node_Y_pred,node_Y) + 5* torch.nn.functional.mse_loss(agg_force_pred,agg_force)
        total_loss += loss.item()
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
    print(f'Epoch {epoch} loss: {total_loss}')
    scheduler.step()
    if epoch % 10 == 0:
        save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,epoch)
save_checkpoint(checkpoint_path,model,optimizer,scheduler,scaler,num_epochs)

`DistributedManager` not initialized already. Initializing now, but this might lead to unexpected errors
/nfs/homedirs/peo/flow_field_dataset/.venv/lib/python3.11/site-packages/modulus/distributed/manager.py:346: UserWarning: Could not initialize using ENV, SLURM or OPENMPI methods. Assuming this is a single process job
  warn(
Could not find valid model file /nfs/homedirs/peo/flow_field_dataset/examples/meshgraphnet/checkpoints/MeshGraphNet.0.0.mdlus, skipping load
Could not find valid checkpoint file, skipping load


Epoch 0 loss: 3.4698176085948944
Epoch 1 loss: 3.2775447219610214
Epoch 2 loss: 3.2215342074632645
Epoch 3 loss: 3.18369397521019
Epoch 4 loss: 3.169101759791374
Epoch 5 loss: 3.1463749557733536
Epoch 6 loss: 3.0906693637371063
Epoch 7 loss: 3.0739634037017822
Epoch 8 loss: 3.0461047887802124
Epoch 9 loss: 2.994960367679596
Epoch 10 loss: 2.9643089920282364
Epoch 11 loss: 2.9510339498519897
Epoch 12 loss: 2.920604959130287
Epoch 13 loss: 2.9158730506896973
Epoch 14 loss: 2.8774108439683914
Epoch 15 loss: 2.83529369533062
Epoch 16 loss: 2.8105750679969788
Epoch 17 loss: 2.799561992287636
Epoch 18 loss: 2.779827669262886
Epoch 19 loss: 2.772114485502243
Epoch 20 loss: 2.757499039173126
Epoch 21 loss: 2.7251996099948883
Epoch 22 loss: 2.7429621517658234
Epoch 23 loss: 2.6938547492027283
Epoch 24 loss: 2.672129362821579
Epoch 25 loss: 2.655619591474533
Epoch 26 loss: 2.639287769794464
Epoch 27 loss: 2.6368962228298187
Epoch 28 loss: 2.6206593364477158
Epoch 29 loss: 2.612465560436249
Epoch

In [35]:
model.to(device)
g=ds_dgl[0].to(device)
g_pred = g.clone().to(device)
ndx, edx = get_node_edge_X(g)
with torch.no_grad():
    y_pred = model(ndx,edx,g)
set_graph_features(g_pred, ndx, edx, y_pred)
ds_dgl.plot_surface(g_pred,"Pressure")

Widget(value='<iframe src="http://localhost:43085/index.html?ui=P_0x7ef9a6f13c50_18&reconnect=auto" class="pyv…

In [36]:
ds_dgl.plot_surface(g,"Pressure")

Widget(value='<iframe src="http://localhost:43085/index.html?ui=P_0x7ef9a17fa450_19&reconnect=auto" class="pyv…

In [37]:
ds_dgl.compute_aggregate_force(g, 1)

tensor([ 0.0140, -0.0021, -0.0022], device='cuda:0')

In [38]:
ds_dgl.compute_aggregate_force(g_pred,1)

tensor([ 0.0089, -0.0023, -0.0045], device='cuda:0')